<a href="https://colab.research.google.com/github/showblue/ai-agent-foundation-labs/blob/main/MealPlanner_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1
Here we draft a meal plan for the week that should align with the user input's parameters.

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import json

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))
GEMINI_MODEL = "gemini-2.5-flash"


def get_completion(prompt,
                   system_prompt="You are a helpful assistant.",
                   json_mode=False):
    model = genai.GenerativeModel(GEMINI_MODEL)
    response = model.generate_content(
        [system_prompt, prompt],
        generation_config={
            "response_mime_type": "application/json" if json_mode else "text/plain"
        }
    )
    return response.text

In [ ]:
schema_string = """
{
  "menu": [
    {
      "day": "",
      "dish": "",
      "ingredients": [],
      "calories": 0,
      "est_cost_usd": 0
    }
  ],
  "total_cost": 0
}
"""

def draft_plan(params):


    system_prompt = """You are a registered dietician who writes weekly dinner plans.
    Output only valid JSON. No markdown backticks."""

    user_prompt = f"""
    <instructions>
 Your task is to create a  {params['days']}–day **dinner** plan for a family of {params['people']} people.

- Assign around {params['daily_calories']} calories *per person* each day
- Keep your estimated total cost less than or equal to ${params['budget_usd']}
- One or more of the family's members have the following allergies: {", ".join(params["allergens"])}.
Do not include any of these allergens in your plan.
IMPORTANT – we already have these ingredients on hand:

{", ".join(params['pantry'])}
Use as MANY of those pantry items as you reasonably can while still meeting the other constraints.
    </instructions>

    <schema>
Return your answer as pure JSON (no markdown, no comments) matching
exactly this schema:
{schema_string}
    </schema>
"""

    raw_json = get_completion(user_prompt,
                              system_prompt,
                              json_mode=True
                              )

    return json.loads(raw_json)

Step 2
Now, a revising step looks at the first draft and generates feedback. There are two types of feedback: fixes and suggestions. Both must be addressed, but suggestions are secondary to fixes, and need only be implemented when they don't conflict with the fixes.

In [ ]:
def critique_plan(plan, params):

    system_prompt = """You are a stern dietary QA inspector.
    Output only valid JSON. No markdown backticks."""

    user_prompt = f"""
    <instructions>
    Here is the proposed plan (raw JSON):
{json.dumps(plan)}

Check for rule violations:

1. The total_cost must be less than or equal to {params['budget_usd']}
2. Each day's calories must be within ±15 % of {params['daily_calories']}
3. NONE of these allergens may appear: {', '.join(params['allergens'])}
4. *Every* pantry item ({', '.join(params['pantry'])}) must appear at
   least once in the week.  Bonus points for using them more often.

Place any violation inside **fixes**.

Then think of nice-to-have tweaks (better variety, seasonal veg, quicker
prep, etc.). Put those ideas in **suggestions**.
</instructions>

<schema>
Reply with JSON matching this schema:
{{"fixes":[], "suggestions":[]}}
Output ONLY the JSON. No markdown backticks, no comments, no extra text.
</schema>
"""

    raw = get_completion(user_prompt,
                         system_prompt,
                         json_mode=True
                         )

    return json.loads(raw)


Step 3
Now the workflow will apply the revisions with the following prompt. We'll also include a function that will create a grocery list after the meal plan is created.

In [ ]:
def revise_plan(plan, fixes, suggestions, params):
    system_prompt = """You are a senior meal planner applying corrections. Output only valid JSON. No markdown backticks."""
    user_prompt = f"""
    <instructions>
Your task is to apply the following corrections to this meal plan:
{json.dumps(plan)}

Here are the mandatory fixes:
{json.dumps(fixes)}

Optional but welcome suggestions:
{json.dumps(suggestions)}
Apply every fix.  Use suggestions only if they don't break the rules.
Return the **updated** plan as plain JSON (same schema, no extra text).
</instructions>

<schema>
Return your answer as pure JSON (no markdown, no comments) matching
exactly this schema:
{schema_string}
</schema>
"""

    raw = get_completion(user_prompt, system_prompt, json_mode=True)
    return json.loads(raw)


def build_grocery_list(plan, pantry):

    system_prompt = """You are a helpful kitchen assistant. Output only valid JSON. No markdown backticks."""

    user_prompt = f"""
    <instructions>
Using the dinner plan below, create a shopping list for one week.
Do **not** include anything already in the pantry:
({', '.join(pantry)}).

Return JSON exactly like:
{{"shopping_list":[{{"item":"","estimated_qty":""}}] }}

Plan JSON:
{json.dumps(plan)}
</instructions>
"""

    raw = get_completion(user_prompt,
                         system_prompt,
                         json_mode=True)

    return json.loads(raw)

Step 4
Finally, let's assemble the workflow.

In [ ]:

user_inputs = {
    "people": 2,
    "days": 7,
    "daily_calories": 2000,
    "allergens": ["peanuts", "shellfish"],
    "budget_usd": 110,
    "pantry": ["rice", "lentils", "frozen spinach"]
}

In [ ]:
MAX_PASSES = 3

def build_plan(params):

    plan = draft_plan(params)
    # If you'd like to see the first draft, uncomment the line below:
    # print(plan)

    for _ in range(MAX_PASSES):

        critique = critique_plan(plan, params)
        # If you'd like to see each critique, uncomment the line below:
        # print(f"Before revisions: {critique}")

        if not critique["fixes"] and not critique["suggestions"]:
            break

        plan = revise_plan(plan, critique['fixes'], critique['suggestions'], params)
        # If you'd like to see each revised plan, uncomment the line below:
        # print(f"After revisions: {plan}")

    return plan


if __name__ == "__main__":

    final_plan = build_plan(user_inputs)

    print("\nFINAL PLAN")
    print(json.dumps(final_plan, indent=2))

    groceries = build_grocery_list(final_plan, user_inputs["pantry"])

    print("\nGROCERY LIST")
    print(json.dumps(groceries, indent=2))


FINAL PLAN
{
  "menu": [
    {
      "day": "Monday",
      "dish": "Hearty Lentil and Spinach Curry with Quinoa",
      "ingredients": [
        "Quinoa (on hand)",
        "Lentils (on hand)",
        "Frozen Spinach (on hand)",
        "1 large onion",
        "3 cloves garlic",
        "1 inch fresh ginger",
        "1 can (14.5 oz) diced tomatoes",
        "1 can (13.5 oz) light coconut milk",
        "2 tbsp curry powder",
        "2 tbsp vegetable oil",
        "Salt and pepper to taste"
      ],
      "calories": 1790,
      "est_cost_usd": 4.7
    },
    {
      "day": "Tuesday",
      "dish": "Chicken and Broccoli Stir-fry with Brown Rice",
      "ingredients": [
        "Brown Rice (on hand)",
        "1 lb boneless, skinless chicken thighs",
        "1 head broccoli",
        "1 large carrot",
        "1 bell pepper (any color)",
        "1 cup sliced mushrooms",
        "1/4 cup low-sodium soy sauce",
        "1 tbsp sesame oil (ensure no peanut contamination)",
        "